Install Library

In [ ]:
pip install nltk

In [ ]:
import pandas as pd 
import re 
import string 
import nltk 
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

Menampilkan dataset

In [ ]:
df = pd.read_csv("/content/ReviewMyIndihome.csv")
df.head()

,userName,score,at,content
0,Asifha Asifha,1,2022-01-27 08:26:00,Selalu dalam perbaikan n ad masalah tambah par...
1,Rojali Susilo,1,2022-01-27 08:25:14,"Verifikasi data katanya 15 menit,, sampai hamp..."
2,dedy wiyono,1,2022-01-27 08:18:01,sudah baik di upgrade jadi turun kwalitas nya....
3,I Wayan Sukadana,3,2022-01-27 08:17:48,Menunlengkapi profile verifikasi lokasi melalu...
4,Memo Andermon.a,1,2022-01-27 08:17:26,Tolong pekerja lapangan selalu pungli


In [ ]:
df = df[['content', 'score']]
df.head()

,content,score
0,Selalu dalam perbaikan n ad masalah tambah par...,1
1,"Verifikasi data katanya 15 menit,, sampai hamp...",1
2,sudah baik di upgrade jadi turun kwalitas nya....,1
3,Menunlengkapi profile verifikasi lokasi melalu...,3
4,Tolong pekerja lapangan selalu pungli,1


Pelabelan

In [ ]:
df = df[df.score != 3]

pd.set_option('mode.chained_assignment', None)
df["labels"] = df["score"].apply(lambda x: 1 if x < 3  else 0) 
df = df.drop("score",axis=1)

df.head()

,content,labels
0,Selalu dalam perbaikan n ad masalah tambah par...,1
1,"Verifikasi data katanya 15 menit,, sampai hamp...",1
2,sudah baik di upgrade jadi turun kwalitas nya....,1
4,Tolong pekerja lapangan selalu pungli,1
5,"Indhihome ruweett, msak mau pindh lokasi aja p...",1


Preprocessing data

In [ ]:
from nltk.corpus import stopwords 
from nltk.tokenize import sent_tokenize, word_tokenize

In [ ]:
def pre_process(Review_Text):

    Review_Text = str(Review_Text).lower()
    Review_Text = re.sub(r"\d+", "", Review_Text)
    Review_Text = Review_Text.translate(str.maketrans("","",string.punctuation))
    Review_Text = Review_Text.strip()
    pisah = Review_Text.split()
    tokens = nltk.tokenize.word_tokenize(Review_Text)
    listStopword =  set(stopwords.words('indonesian'))
 
    removed = []
    for t in tokens:
      if t not in listStopword:
        removed.append(t)

    return Review_Text

df['content'] = df['content'].apply(lambda x:pre_process(x))
df.head()

,content,labels
0,selalu dalam perbaikan n ad masalah tambah par...,1
1,verifikasi data katanya menit sampai hampir ...,1
2,sudah baik di upgrade jadi turun kwalitas nyak...,1
4,tolong pekerja lapangan selalu pungli,1
5,indhihome ruweett msak mau pindh lokasi aja pk...,1


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
cv.fit(df['content'])
X = cv.transform(df['content'])

y = df['labels']

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.80)

In [ ]:
for c in [0.01, 0.05, 0.25, 0.5, 1]:
    
    lr = LogisticRegression(C=c)
    lr.fit(X_train, y_train)
    print('Accuracy for C=%s: %s'
         % (c, accuracy_score(y_test, lr.predict(X_test))))

Accuracy for C=0.01: 0.908152652212177
Accuracy for C=0.05: 0.9204736378910313


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


Accuracy for C=0.25: 0.9275142011360908


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


Accuracy for C=0.5: 0.9267941435314825
Accuracy for C=1: 0.9267141371309705


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [ ]:
for c in [0.01, 0.05, 0.25, 0.5, 1]:
    
    sv = SVC(C=c)
    sv.fit(X_train, y_train)
    print('Accuracy for C=%s: %s'
         % (c, accuracy_score(y_test, sv.predict(X_test))))

Accuracy for C=0.01: 0.8269461556924554
Accuracy for C=0.05: 0.8937515001200096
Accuracy for C=0.25: 0.9170333626690135
Accuracy for C=0.5: 0.9227938235058805
Accuracy for C=1: 0.9270341627330186


In [ ]:
final_model_lr = LogisticRegression(C=1)
final_model_lr.fit(X, y)
print('Final Model Accuracy: %s' %accuracy_score(y_test, final_model_lr.predict(X_test)))

Final Model Accuracy: 0.9583166653332267


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [ ]:
final_model_sv = SVC(C=1)
final_model_sv.fit(X, y)
print('Final Model Accuracy: %s' %accuracy_score(y_test, final_model_sv.predict(X_test)))

Final Model Accuracy: 0.9580766461316905
